*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will create a book recommendation algorithm using **K-Nearest Neighbors**.

You will use the [Book-Crossings dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/). This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users. 

After importing and cleaning the data, use `NearestNeighbors` from `sklearn.neighbors` to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures distance to determine the “closeness” of instances.

Create a function named `get_recommends` that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

This code:

`get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")`

should return:

```
[
  'The Queen of the Damned (Vampire Chronicles (Paperback))',
  [
    ['Catch 22', 0.793983519077301], 
    ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479], 
    ['Interview with the Vampire', 0.7345068454742432],
    ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
    ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
  ]
]
```

Notice that the data returned from `get_recommends()` is a list. The first element in the list is the book title passed in to the function. The second element in the list is a list of five more lists. Each of the five lists contains a recommended book and the distance from the recommended book to the book passed in to the function.

If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

The first three cells import libraries you may need and the data to use. The final cell is for testing. Write all your code in between those cells.

In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2022-07-19 20:27:27--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  61.7MB/s    in 0.4s    

2022-07-19 20:27:28 (61.7 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [4]:
# add your code here - consider creating a new cell for each section of code
# print(df_books.head()) # isbn, title, author
# print(df_ratings.head()) # user, isbn, rating
df_temp = pd.DataFrame(df_ratings.groupby('isbn')['rating'].count().sort_values(ascending=False))
df_f_books = df_temp[df_temp['rating']>100]
df_temp = pd.DataFrame(df_ratings.groupby('user')['rating'].count().sort_values(ascending=False))
df_f_ratings = df_temp[df_temp['rating']>200]
# df_f_books.head()
# df_f_ratings.head()
df_temp = pd.merge(df_ratings, df_books, how='inner', on='isbn')
# df_temp.head()
df_temp = pd.merge(df_temp, df_f_ratings, on='user')
# df_temp.sort_values('user').head()


In [5]:
df = pd.merge(df_temp, df_f_books, on='isbn')
df.head()

,user,isbn,rating_x,title,author,rating_y,rating
0,6543,0316666343,0.0,The Lovely Bones: A Novel,Alice Sebold,441,1295
1,23768,0316666343,7.0,The Lovely Bones: A Novel,Alice Sebold,1708,1295
2,28523,0316666343,0.0,The Lovely Bones: A Novel,Alice Sebold,397,1295
3,81977,0316666343,7.0,The Lovely Bones: A Novel,Alice Sebold,320,1295
4,98391,0316666343,9.0,The Lovely Bones: A Novel,Alice Sebold,5891,1295


In [6]:
# df['average_rating'] = df.rating_y/df.rating
# df.head()
# df.astype({'isbn':str, 'title':str, 'author':str})
# df.info()
# df.isnull().sum()

df1 = df.drop(columns=['rating','rating_y','author','isbn'], axis=1)
df1 = df1.pivot_table(index=['title'],columns=['user'],values='rating_x').fillna(0)
# df1.head()
matrix = csr_matrix(df1.values)

In [8]:
model = NearestNeighbors(n_neighbors=5, p=2, metric='cosine')
model.fit(matrix)

NearestNeighbors(metric='cosine')

[(0.8016211, "I'll Be Seeing You"), (0.77085835, 'The Weight of Water'), (0.7699411, 'The Surgeon'), (0.7677075, 'I Know This Much Is True'), (0.7230184, 'The Lovely Bones: A Novel')]


In [21]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  distances, indices = model.kneighbors([df1.loc[book].values], n_neighbors=6)
  out = []
  for d,n in zip(distances[0],indices[0]):
    out.append((d,df1.index[n]))
  out.sort(reverse=True)
  recommended_books = [ book, 
                       [
                        [out[0][1], out[0][0]],
                        [out[1][1], out[1][0]],
                        [out[2][1], out[2][0]],
                        [out[3][1], out[3][0]]
                        ]
                       ]
  return recommended_books

Use the cell below to test your function. The `test_book_recommendation()` function will inform you if you passed the challenge or need to keep trying.

In [23]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075]]]
You passed the challenge! 🎉🎉🎉🎉🎉
